## Setup
1. In `cd NEMO`
1. Run YAP API server `./yap api` (if you specify a port, change it in `config.py`)
1. `pip install -r requirements.txt`
1. install bclm https://github.com/OnlpLab/bclm
1. Run NEMO API server `uvicorn api_main:app --reload --port 8090`
1. Have a look at the swagger OpenAPI documentation by opening http://localhost:8090/docs in your browser

In [104]:
import requests

In [119]:
requests.get('http://localhost:8090/').json()

{'message': 'Please specify command in URL path.',
 'available_commands': ['run_ner_model',
  'multi_align_hybrid',
  'multi_to_single',
  'morph_yap',
  'morph_hybrid',
  'morph_hybrid_align_tokens']}

In [120]:
texts = [
    'מלך השערים במונדיאל 2006 היה מירוסלב קלוזה.',
    "עשרות אנשים מגיעים מתאילנד לישראל.\nתופעה זו התבררה אתמול בוועדת העבודה והרווחה של הכנסת."
]

## Run NER Model

In [122]:
%%time 
payload = { 'sentences': texts[1],
            'model_name': 'token-multi', # / 'token-single' / 'morph' 
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/run_ner_model', params=payload).json()
list(zip(res['tokenized_text'][0], res['nemo_predictions'][0])), list(zip(res['tokenized_text'][1], res['nemo_predictions'][1]))

CPU times: user 9.17 ms, sys: 2.86 ms, total: 12 ms
Wall time: 89.7 ms


([('עשרות', 'O'),
  ('אנשים', 'O'),
  ('מגיעים', 'O'),
  ('מתאילנד', 'O^S-GPE'),
  ('לישראל', 'O^S-GPE'),
  ('.', 'O')],
 [('תופעה', 'O'),
  ('זו', 'O'),
  ('התבררה', 'O'),
  ('אתמול', 'O'),
  ('בוועדת', 'O^B-ORG'),
  ('העבודה', 'I-ORG^I-ORG'),
  ('והרווחה', 'I-ORG^I-ORG^E-ORG'),
  ('של', 'O'),
  ('הכנסת', 'B-ORG^E-ORG'),
  ('.', 'O')])

### multi_to_single (get token-single predictions using a token-multi model)

In [123]:
%%time
payload = { 'sentences': texts[1],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_to_single', params=payload).json()
res

CPU times: user 10.8 ms, sys: 0 ns, total: 10.8 ms
Wall time: 90.9 ms


{'tokenized_text': [['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.']],
 'nemo_multi_predictions': [['O', 'O', 'O', 'O^S-GPE', 'O^S-GPE', 'O'],
  ['O',
   'O',
   'O',
   'O',
   'O^B-ORG',
   'I-ORG^I-ORG',
   'I-ORG^I-ORG^E-ORG',
   'O',
   'B-ORG^E-ORG',
   'O']],
 'single_predictions': [['O', 'O', 'O', 'S-GPE', 'S-GPE', 'O'],
  ['O', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'E-ORG', 'O', 'S-ORG', 'O']]}

## MD + NER

### multi_align_hybrid

In [124]:
%%time
payload = { 'sentences': texts[1],
            #'model_name': 'token-multi',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/multi_align_hybrid', params=payload).json()
res

CPU times: user 11.5 ms, sys: 0 ns, total: 11.5 ms
Wall time: 235 ms


{'tokenized_text': [['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.']],
 'nemo_multi_predictions': [['O', 'O', 'O', 'O^S-GPE', 'O^S-GPE', 'O'],
  ['O',
   'O',
   'O',
   'O',
   'O^B-ORG',
   'I-ORG^I-ORG',
   'I-ORG^I-ORG^E-ORG',
   'O',
   'B-ORG^E-ORG',
   'O']],
 'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=

In [125]:
print(res['md_lattice'])

0	1	עשרות	עשר	CDT	CDT	gen=F|num=P	1
1	2	אנשים	איש	NN	NN	gen=M|num=P	2
2	3	מגיעים	הגיע	BN	BN	gen=M|num=P|per=A	3
3	4	מ	מ	PREPOSITION	PREPOSITION	_	4
4	5	תאילנד	תאילנד	NNP	NNP	gen=F|num=S	4
5	6	ל	ל	PREPOSITION	PREPOSITION	_	5
6	7	ישראל	ישראל	NNP	NNP	gen=F|num=S	5
7	8	.	_	yyDOT	yyDOT	_	6

0	1	תופעה	תופעה	NN	NN	gen=F|num=S	1
1	2	זו	זו	PRP	PRP	gen=F|num=S|per=3	2
2	3	התבררה	התברר	VB	VB	gen=F|num=S|per=3|tense=PAST	3
3	4	אתמול	אתמול	RB	RB	_	4
4	5	ב	ב	PREPOSITION	PREPOSITION	_	5
5	6	וועדת	ועדה	NNT	NNT	gen=F|num=S	5
6	7	ה	ה	DEF	DEF	_	6
7	8	עבודה	עבודה	NN	NN	gen=F|num=S	6
8	9	ו	ו	CONJ	CONJ	_	7
9	10	ה	ה	DEF	DEF	_	7
10	11	רווחה	רווחה	NN	NN	gen=F|num=S	7
11	12	של	של	POS	POS	_	8
12	13	ה	ה	DEF	DEF	_	9
13	14	כנסת	כנסת	NNP	NNP	gen=F|num=S	9
14	15	.	_	yyDOT	yyDOT	_	10




### morph_yap

In [126]:
%%time
payload = { 'sentences': texts[1],
            #'model_name': 'morph',
            #'tokenized': True,
          }

res = requests.get('http://localhost:8090/morph_yap', params=payload).json()
res

CPU times: user 11.9 ms, sys: 455 µs, total: 12.4 ms
Wall time: 283 ms


{'tokenized_text': [['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.']],
 'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|gen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=F|num=

### morph_hybrid
The following models got the best results in our experiments.


In [127]:
%%time
payload = { 'sentences': texts[1],
            #'multi_model_name': 'token-multi',
            #'morph_model_name': 'morph',
            #'tokenized': False,
            # 'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()
res

CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 245 ms


{'tokenized_text': [['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.']],
 'nemo_multi_predictions': [['O', 'O', 'O', 'O^S-GPE', 'O^S-GPE', 'O'],
  ['O',
   'O',
   'O',
   'O',
   'O^B-ORG',
   'I-ORG^I-ORG',
   'I-ORG^I-ORG^E-ORG',
   'O',
   'B-ORG^E-ORG',
   'O']],
 'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=

### morph_hybrid_align_tokens

In [128]:
%%time
payload = { 'sentences': texts[1],
            #'tokenized': False,
            #'align_tokens': False,
          }

res = requests.get('http://localhost:8090/morph_hybrid_align_tokens', params=payload).json()
res

CPU times: user 11.2 ms, sys: 0 ns, total: 11.2 ms
Wall time: 358 ms


{'tokenized_text': [['עשרות', 'אנשים', 'מגיעים', 'מתאילנד', 'לישראל', '.'],
  ['תופעה',
   'זו',
   'התבררה',
   'אתמול',
   'בוועדת',
   'העבודה',
   'והרווחה',
   'של',
   'הכנסת',
   '.']],
 'nemo_multi_predictions': [['O', 'O', 'O', 'O^S-GPE', 'O^S-GPE', 'O'],
  ['O',
   'O',
   'O',
   'O',
   'O^B-ORG',
   'I-ORG^I-ORG',
   'I-ORG^I-ORG^E-ORG',
   'O',
   'B-ORG^E-ORG',
   'O']],
 'ma_lattice': '0\t1\tעשרות\tעשר\tCDT\tCDT\tgen=F|num=P\t1\n0\t1\tעשרות\tעשר\tCD\tCD\tgen=F|num=P\t1\n1\t2\tאנשים\tהנשים\tVB\tVB\tgen=F|gen=M|num=S|per=1|tense=FUTURE\t2\n1\t2\tאנשים\tאיש\tNN\tNN\tgen=M|num=P\t2\n2\t3\tמגיעים\tהגיע\tVB\tVB\tgen=M|num=P|per=A|tense=BEINONI\t3\n2\t3\tמגיעים\tהגיע\tBN\tBN\tgen=M|num=P|per=A\t3\n3\t4\tמ\tמ\tPREPOSITION\tPREPOSITION\t_\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=P|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNN\tNN\tgen=M|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=F|num=S\t4\n3\t5\tמתאילנד\tמתאילנד\tNNP\tNNP\tgen=

## Working with sequence labels
`iobes` can be used to parse the predictions (`pip install iobes`)

In [129]:
import iobes
import pandas as pd

In [130]:
payload = { 'sentences': '\n'.join(texts),}

res = requests.get('http://localhost:8090/morph_hybrid', params=payload).json()

In [131]:
def get_ents(toks, labels):
    ents = []
    for i, (sl, sm) in enumerate(zip(labels, toks)):
        spans = iobes.parse_spans_iobes(sl)
        for span in spans:
            text = ' '.join(sm[span.start:span.end])
            ents.append({ 'sent_id': i, 
                          'text': text, 
                          'cat': span.type})
    return pd.DataFrame(ents)

In [132]:
labels = res['nemo_morph_predictions']
morphs = res['morph_segmented_text']
ents_hyb = get_ents(morphs, labels)
ents_hyb

,sent_id,text,cat
0,0,מירוסלב קלוזה,PER
1,1,תאילנד,GPE
2,1,ישראל,GPE
3,2,וועדת ה עבודה ו ה רווחה,ORG
4,2,ה כנסת,ORG


In [133]:
labels = res['morph_aligned_multi_predictions']
morphs = res['morph_segmented_text']
ents_align = get_ents(morphs, labels)
ents_align

,sent_id,text,cat
0,0,מירוסלב קלוזה,PER
1,1,תאילנד,GPE
2,1,ישראל,GPE
3,2,וועדת ה עבודה ו ה רווחה,ORG
4,2,ה כנסת,ORG
